# Under construction ... starting from here

In [7]:
! pip install transformers[torch] datasets evaluate swifter accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=67c698c7e4e0c939461d02ee6d599e119b611cd7ba935444839b278a052f5ce4
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa37

In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
# load standard modules/libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# load special modules/libraries
import os
import warnings
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from collections import Counter
import string
import re
from tqdm  import tqdm

# load pytorch modules/libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader

from sklearn.metrics import f1_score, roc_auc_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from torch.utils.data import Dataset, DataLoader

class GFMDataset(Dataset):

  def __init__(self, description, targets, tokenizer, max_len):
    self.description = description
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.description)

  def __getitem__(self, item):
    description = str(self.description[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      description,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
      truncation = True
    )

    return {
      'review_text': description,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [11]:
from transformers import AutoTokenizer
MAX_LEN = 500
BATCH_SIZE = 20

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def load_and_preprocess_dataset(file_path):
    # Load dataset
    raw_df = pd.read_csv(file_path)

    # Subsample 2500 samples for each class where success == 1 and success == 0
    df = pd.concat([
        raw_df[raw_df['success'] == 1].sample(n=2500, random_state=42),
        raw_df[raw_df['success'] == 0].sample(n=2500, random_state=42)
    ], ignore_index=True)

    # Display class distribution
    class_distribution = df['success'].value_counts()
    print("Class Distribution:")
    print(class_distribution)

    # First, split the data into training and temporary data (temp_data)
    df_temp, df_test = train_test_split(df, test_size=0.2, random_state=42)

    # Now, split the temporary data into training and validation data
    df_train, df_val = train_test_split(df_temp, test_size=0.25, random_state=42)

    train_data = GFMDataset(description=df_train['Description'].to_numpy(),
                                    targets=df_train['success'].to_numpy(),
                                    tokenizer=tokenizer,
                                    max_len=MAX_LEN)
    train_data_loader = DataLoader(train_data, batch_size = BATCH_SIZE, num_workers = 4) #modify num_works as needed

    val_data = GFMDataset(description=df_val['Description'].to_numpy(),
                              targets=df_val['success'].to_numpy(),
                              tokenizer=tokenizer,
                              max_len=MAX_LEN)
    val_data_loader = DataLoader(val_data, batch_size=BATCH_SIZE, num_workers=4)  # modify num_works as needed

    test_data = GFMDataset(description=df_test['Description'].to_numpy(),
                                targets=df_test['success'].to_numpy(),
                                tokenizer=tokenizer,
                                max_len=MAX_LEN)
    test_data_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=4)
    return train_data_loader, val_data_loader, test_data_loader

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import wandb

# Load and preprocess dataset
train_loader, valid_loader, test_loader = load_and_preprocess_dataset("/content/gdrive/MyDrive/ECE1786_Project/processed_en_dataset.csv")

# Load evaluation functions
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
auc = evaluate.load("roc_auc")
recall = evaluate.load("recall")
precision = evaluate.load("precision")

def compute_metrics(eval_pred):
    # predictions = eval_pred.predictions
    # labels = eval_pred.label_ids
    # # Assuming your model outputs logits, use softmax to get probabilities
    # probabilities = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    # predicted_labels = np.argmax(probabilities, axis=1)
    # # Compute different metrics
    # acc = accuracy.compute(predictions=predicted_labels, references=labels)["accuracy"]
    # f1_val = f1.compute(predictions=predicted_labels, references=labels)["f1"]
    # # auc_val = auc.compute(predictions={"prediction_scores": probabilities[:, 1]}, references=labels)  # Assuming binary classification
    # auc_val = roc_auc_score(labels, probabilities[:, 1])
    # recall_val = recall.compute(predictions=predicted_labels, references=labels)["recall"]
    # precision_val = precision.compute(predictions=predicted_labels, references=labels)["precision"]
    # # Calculate loss using cross-entropy
    # loss = F.cross_entropy(torch.tensor(predicted_labels), torch.tensor(labels).float()).item()
    # return {
    #     "loss": loss,
    #     'accuracy': acc,
    #     'f1_score': f1_val,
    #     'auc': auc_val,
    #     'recall': recall_val,
    #     'precision': precision_val
    # }
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    # Assuming your model outputs logits, use softmax to get probabilities
    probabilities = F.softmax(torch.tensor(predictions), dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    # Compute different metrics
    acc = accuracy.compute(predictions=predicted_labels.numpy(), references=labels)["accuracy"]
    f1_val = f1.compute(predictions=predicted_labels.numpy(), references=labels)["f1"]
    auc_val = roc_auc_score(labels, probabilities[:, 1])
    recall_val = recall.compute(predictions=predicted_labels.numpy(), references=labels)["recall"]
    precision_val = precision.compute(predictions=predicted_labels.numpy(), references=labels)["precision"]
    # Calculate loss using cross-entropy
    loss = F.cross_entropy(torch.tensor(predictions), torch.tensor(labels).long()).item()
    return {
        "loss": loss,
        'accuracy': acc,
        'f1_score': f1_val,
        'auc': auc_val,
        'recall': recall_val,
        'precision': precision_val
    }

id2label = {0: "UNSUCCESSFUL", 1: "SUCCESSFUL"}
label2id = {"UNSUCCESSFUL": 0, "SUCCESSFUL": 1}

# Initialize W&B
wandb.init()

# Load T5 model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/ECE1786_Project/t5-output/",
    learning_rate=1e-5,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    report_to='wandb',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_loader.dataset,
    eval_dataset=valid_loader.dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


Class Distribution:
1    2500
0    2500
Name: success, dtype: int64


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


ValueError: ignored